In [22]:
#https://github.com/neolithian/banglanews
!pip install banglanews
!pip install bnunicodenormalizer
!pip install pandarallel

In [21]:
!mkdir -p /tmp/prothom_alo_data

!mkdir -p /kaggle/working/bn_clean

In [23]:
#prothom_alo
from banglanews import prothomalo

objScraper = prothomalo.scraper('2022-01-01','2022-10-02','/tmp/prothom_alo_data/')

objScraper

objScraper.PrintContents()

Calculating batch process size...
Estimated total URL found in the given criteria: 214
Calculating batch process size done.
Process started...
URL found : 214
Process done.
Time elapsed: 0:00:03


In [6]:
import glob
import os 
import pandas as pd
from tqdm import tqdm
import re
#normalization
from pandarallel import pandarallel
from bnunicodenormalizer import Normalizer 
pandarallel.initialize(progress_bar=False,nb_workers=8)
tqdm.pandas()
bnorm=Normalizer()

# Set environment variables
import warnings
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’…,<>.‚/?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [8]:
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 

In [19]:
ls = glob.glob('/tmp/prothom_alo_data/*')
ls_td = [tex for tex in ls if len(tex)<20]
if len(ls_td) > 0:
    ls.remove(ls_td[0])

In [ ]:
f_name = 1
for txt in tqdm(ls):
    with open(txt) as f:
        lines = f.readlines()
        if (len(lines)) == 6:
            lines = lines[5]
            lines = re.split(r'[,;!?।]', lines)
            if len(lines) > 5:
                df = pd.DataFrame()
                df['sentence'] = lines
                df = df.dropna(how='all')
                df['sentence'] = df['sentence'].apply(lambda x : remove_punctuations(x))
                df["sentence"] = df["sentence"].parallel_apply(lambda x:normalize(x))
                df.to_csv(f'/kaggle/working/bn_clean/{f_name}.csv',index=False)
                f_name += 1

 55%|██████████████████████████████████████████████████████████████████████████████▌                                                               | 37171/67226 [4:03:08<2:39:50,  3.13it/s]